## Market Making v.1
Fixed up closing positions. Instead of going to the market to close positions, we will instead send quotes and wait until someone executes on us to close out.

![Image Title](images/MM1.png)

## Market Making v.2
There are times that our profit drops too far down. In theory, market making, if both bid and asks are executed, should only make money. Issues could arise from the fact that a bot lifts our ask (leaving us in a short position) and then the asset price increases. So we must also increase our bid and ask, then we must quote higher bid prices (which end up being higher than our intial ask price), when a bot hits our bid, we must buy at a higher price than we sold at intially --> a loss.
- Added in functionality to only trade when the spread of the product was above a hard coded spread ($6 units)

![Image Title](images/MM2.png)

## Market Making v.3
An issue is that while we have an option position, our code does not continue to market make. Instead it continues to quote orders to try to close out of the position to lock in profit. This results in us missing opportunties to continue to trade in the market. 
- Added in logic to continue to market make with the remaining position (position limit - current position) while we try to close out.

![Image Title](images/MM3.png)

## Market Making v.4
Before we hard coded the spread. This wouldn't be scalable in the future as we would need to determine and hard code the spread values. Thus we decided to code a way to find the average spread of the last 20 time periods. Then we would trade if and only if our current_spread > avg_spread*scale_factor. Where avg_spread is the average spread in the last 20 (changeable) time periods and scale factor is a multiplier. Currently we have best successes with scale factors of 0.8 - 1. Lower scale factors mean more trades are eligbile and and thus more trades, but potentially bad trades. Higher scale factors mean less trades, but more potential profit for each trade.
- Added in moving average for product spreads --> used to determine the required spread to market make

![Image Title](images/MM4.png)
![Image Title](images/MM4.1.png)

## Market Making v.5 
Before this, whenever we quote prices to market make, we would always beat the market by $1 to ensure we had the best prices and all participants would trade with us first. However this default, while netting us more trades, reduced our profit due to the smaller spread. So we decided to change our quote depending on if there was high or low amounts of volume for the best bid and ask.

- If there is low best bid/best ask volume, then instead of beating the price by $1, we can just match it with greater volume. Locking in more profit if it manages to execute (which it had a higher chance to due to low volume)

![Image Title](images/MM5.png)

## Market Making v.6
For the last few versions, if the spread of the market was outside of our required spread (required spread was an average of the last 40 spreads) we would just not send an order to the market as the reward was not justifiable enough for us to risk. However we then realised our of 4000 chances (2000 iterations, 2 products), we were not sending orders ~850 times. We were missing out on too many chances to make more profit. 
- If the market's spread is not large enough for us to comfortably market make prices (current_spread was not >= required_spread), then we will still send market making orders except the orders will conform to our required_spread at the detriment of not being the best bid/ask in the market. 
    - Best_bid = 10, Best_ask = 14 --> current_spread = 4
    - required_spread = 5 (historically)
    - Thus we cannot market make the best orders as the spread is too small. However we will still send orders at either 10-15 or 9-14 depending on the volume of the best bid/best ask.
        - To determine if we should MM at 10-15 or 9-14 placed the "better" order at the bid/ask with the lower volume as it would more likely be filled.

It is interesting to note that, we almost have perfect profits for AMETHYST, from observing the graph we can see that our profits are either increasing or remaining the same, never decreasing. 

![Image Title](images/MM6.png)
![Image Title](images/MM6.1.png)

## Market Making v.7
Modified the code to include an EMA and Standard Deviation. Then if the price was greater than or less than the 1.5*std_dev then it would be at outlier and we would expect the next time stamp to increase/decrease back to the trend. Thus if we expect the price to increase, we would only quote bids, if we expect it to decrease, we would only quote asks.

![Image Title](images/MM7.png)

## Market Making v.8

**We switched over from using EMA/Std to Linear Regression/Std from this point onwards**

By using a trend predictor with linear regression of the last 15 points as our guideline, we were able to identify that 81.15% of the time, the next point would follow our linear regression with an error tolerance of $\pm$ 1.5 $\times$ sd calculated from the last 15 points. This implies that 18.85% of the times there is an opportunity to take advantage of outliers (when the market doesn't follow the trend) assuming that our trend predictor is 100% accurate (from visually sampling a small portion the data, we assumed this is correct).

Using this method, we can perform the same strategy as MMv.7 of taking advantage of outliers (only quoting bids/asks) but with more confidence that our trend predictor is correct.

![Image Title](images/MM8.png)

## Market Making v.9

By adjusting the error tolerance of $\pm$ 1.5 $\times$ sd to $\pm$ 0.9 $\times$ sd, we achieve a greater profit. 

It was at this point we realised that to testing all of the different combinations of the no. datapoints kept for midprice/prices, standard deviation multiplier, volume cap... would be necessary to maximise profit, but next to impossible. Thus we began to code up an program that would automatically make changes to these parameters and save it as a new file to submit.

![Image Title](images/MM9.png)

## Market Making v.10

Having developed confidence in our predictor to make accurate calls, we can now take advantage of the fact when there is a discrepency in market price and our expected price.

As such, we can close out of our positions at a more aggressive price than we previously could by beating the market by putting prices that are at the mid price of the spread. 
- Before, to try to close out of our position we would only continue to market make the best bid/ask prices, hoping for someone else to buy/sell our order. 
- However with the implementation of the linear regression, we can be more confident in the true price of the asset, giving us the ability to quote prices very closely around the mid price resulting in better prices and faster execution. 
    - We found that by taking a more aggressive approach, rounding the mid price up for quoting bids and rounding the mid price down for quoting asks, gives us the best results.
- On top of the above (sending competitive bid/ask), if we find an outlier, we will also go to the market and determine if we can buy/sell at market price. Even though we would be buying for more than current mid price and selling for less than current mid price, if we can buy for less than the theoretical value of the next time stamp we will do so.


![Image Title](images/MM10.png)

## Market Making v.11

In our previous version, we were still trying to close out of our positions at every time stamp, even if it was an outlier. 
- Closing out only took into account if we were long or short. 
    - If we were currently long, we should send orders to short to try to close out. 
- At times, this was detrimental to our profit as if we were long AND the mid price was less than the lower bound (and thus we expect price to increase), our code would send orders to short. The next time stamp price would increase and we would lose money. 

Now, we will ONLY try to close out of any open positions during the "market making" periods. When the price range falls within the expected price. Other times, during outliers, our goal is not to close out of positions but rather to buy/sell at lower/higher than fair value to generate profit.

Additionally instead of quoting outlier bid/ask around the mid price, we simply went to the market and beat their prices by $1. It didn't make sense to be overly aggressive and quote around the mid price and beat the best bid by for example $2, beating by $1 would have been enough to secure priority and increase profits.

![Image Title](images/MM11.png)